#End-to-End Multi-class Dog Breed Classification

This notebook builds an end-to-end multi-class image classifier using Tensorflow 2.0 and TensorFlow Hub

##1. Problem

Identifying the breed of a dog given an image of a dog.

##2. Data

The data we're using is from Kaggle's dog breed identification competition.

https://www.kaggle.com/c/dog-breed-identification/data

##3. Evalution

The evalution is a file with prediction probabilities for each dog breed of each test image.

https://www.kaggle.com/c/dog-breed-identification/overview/evaluation

##4. Features

Some information about data:
* We're dealing with images(Unstructured data) so it's probably best we use deep learning/transfer learning
* There are 120 breeds of dog (this means therer are 120 diffrent classes).
* There are aroung 10,000+ images in training set (have labels)
* There are around 10,000+ images in test set(no labels)

In [ ]:
#Unzip the uploaded data into Google Drive
#!unzip "/content/drive/MyDrive/Dog-Vision/dog-breed-identification.zip" -d "/content/drive/MyDrive/Dog-Vision/"

### **Get our workspace ready**
* Import TensorFlow
* Import Tesnsorflow Hub
* Make sure we're using GPU

In [ ]:
#Import necessary tools

import tensorflow as tf
import tensorflow_hub as hub
print("TF version : ",tf.__version__)
print("TF hub version : ",hub.__version__)

#Check for GPU availability
print("GPU", "available (YESSSSS!!!!)" if tf.config.list_physical_devices("GPU") else "not available :(")

## Getting our data ready (turning into Tensors)

With  all machine learning models, our data has to be in numerical format.so that's whar we'll be doing first. Turning out images into Tensors.

Let's start by accessing out data and check out the labels.

In [ ]:
import pandas as pd
labels_csv=pd.read_csv("/content/drive/MyDrive/Dog-Vision/labels.csv")
print(labels_csv.describe())

In [ ]:
labels_csv.head()

In [ ]:
#how many images are therre of each breed?

labels_csv["breed"].value_counts().plot(kind="bar", figsize=(20,10))

In [ ]:
labels_csv["breed"].value_counts().median()

In [ ]:
#Let's view an image
from IPython.display import Image
Image("/content/drive/MyDrive/Dog-Vision/train/000bec180eb18c7604dcecc8fe0dba07.jpg")

## Getting images and their labels

Let's get alist of all of our images file pathnames.

In [ ]:
#Creat pathnames from image ID's
filenames = ["/content/drive/MyDrive/Dog-Vision/train/" + fname + ".jpg" for fname in labels_csv["id"]]

In [ ]:
#Check the first 10
filenames[:10]

In [ ]:
#Check wheather number of filenames match number of actual image files
import os

if len(os.listdir("/content/drive/MyDrive/Dog-Vision/train")) == len(filenames):
  print("Filenames Match actual amount of files..")
else:
  print("Filename do not match actual amount of files, check the target directory") 

In [ ]:
#One more check 
Image(filenames[9000])
#print(labels_csv["breed"][9000])

Since we've now got out training filepaths in a listr, let's prepare our label

In [ ]:
import numpy as np
labels = labels_csv["breed"]
labels = np.array(labels)
labels

In [ ]:
len(labels)

In [ ]:
#See if number of labewls matches the number of filenames
if len(labels)==len(filenames):
  print("Matches")
else:
  print("Does'nt match")

In [ ]:
#Find the unique label values
Unique_breeds = np.unique(labels)

In [ ]:
len(Unique_breeds)

In [ ]:
#Turn every label into a boolean array
boolean_labels = [label==Unique_breeds for label in labels]
boolean_labels

In [ ]:
#Example : Turing boolean array into integers
print(labels[0])# original label
print(np.where(Unique_breeds== labels[0]))# index where label occurs
print(boolean_labels[0].argmax())#index where label occurs in boolean array
print(boolean_labels[0].astype(int))# there will be a 1 where the sample label occurs

In [ ]:
type(boolean_labels)

##Creating our own validation set
Since the dataset from Kaggle doesn't come with a validation set, we're going to create our own.

In [ ]:
#Setup X & y variables
X=filenames
y=boolean_labels

We're going to start off experimenting with ~1000 images and increase as needed

In [ ]:
#Set number of images to use for experimenting
NUM_IMAGES = 1000 #@param {type:"slider", min:1000, max:10000, step:1000}

In [ ]:
# splite our data into train and validation sets
from sklearn.model_selection import train_test_split

# split them into training and validation of total size NUM_IMAGES
X_train, X_valid, y_train, y_valid = train_test_split(X[:NUM_IMAGES], y[:NUM_IMAGES], test_size=0.2, random_state=42)

len(X_train), len(y_train), len(X_valid), len(y_valid)

In [ ]:
# Let's have geez at the training data
X_train[:2], y_train[:2]

## Preprocessing Images (turing images into Tensors)
 
To preprocess our images into Tensors we're goin gto write a function which does a few things:

1. Take an image filepath as input
2. Use Tensorflow to read the file and save it to a variabel `image`
3. Turn our `image` (a jpg) into Tensors
4. Normalize our image (convert color channel values from 0-255 to 0-1)
4. Resize the `image` to be a shape of (224,224)
5. Return the modified `image`

Before we do, Let's see what importing an image looks like...

In [ ]:
# COnvert image to Numpy array
from matplotlib.pyplot import imread
image= imread(filenames[2000])
image.shape

In [ ]:
image.min(), image.max()

In [ ]:
image[:1]

In [ ]:
#turn image into Tensor
tf.constant(image[:1])

Now we've seen what an image looks like Tensor, let's make function to preprocess them .

In [ ]:
#Define image size
IMG_SIZE = 224

#Create a function for preprocessing images
def process_image(image_path, img_size=IMG_SIZE):
  """
  Take an image file path, image size and turns the image into a Tensor.
  """

  #Read in an image file
  image = tf.io.read_file(image_path)
  
  #Turn the jpeg image into numerical Tensor with 3 color channels (Red, Green and Blue)
  image = tf.image.decode_jpeg(image, channels=3)

  #Convert the color channel value from 0-255 to 0-1 values
  image = tf.image.convert_image_dtype(image, tf.float32)

  #Resize the image to our desired value (224,224)
  image = tf.image.resize(image, size=[img_size, img_size])

  return image

## Turning data into batches

Why turn our data into batches?

Let's say you're trying to process 10,000+ images in one go....
they all might not fit into memory

So that's why we do about 32 (this is batch size) images at a time (you can manualy adjust the batch size if need to be).

In order to use tensorflow effectively, we need our data in the form of Tensor tuple which look like this:
`(image,label)`

In [ ]:
#Create a simple function to return a tuple (image, label)
def get_image_label(image_path, label):

  """
  Takes an image file path name and the assoscaited label,
  processes the image and returns a tuple of(image, label).
  """

  image= process_image(image_path)
  return image, label

Now we've got a way to turn our data into tuples of Tensors in the form: `(image,label)`, let's make a function to turn all of our data (X & y) into batches!

In [ ]:
# Define the batch size, 32 is good start
BATCH_SIZE = 32

#Create a function to turn data into batches
def create_data_batches(X, y=None, batch_size=BATCH_SIZE, valid_data=False, test_data=False):
  """
  Create batches of data out of images (X) and label (y) pairs.
  Shuffles the data if it's training data but doesn't shuffle if it's validation data.
  Also accept test data as input (no label).
  """
  # If the data is test dataset, we probably don't have labels
  if test_data:
    print("Creating test data batcehs...")
    data=tf.data.Dataset.from_tensor_slices((tf.constant(X))) #only filepaths (no labels)
    data_batch = data.map(process_image).batch(BATCH_SIZE)
    return data_batch

  elif valid_data:
    print("Creating Valid data batches...")
    data=tf.data.Dataset.from_tensor_slices((tf.constant(X), # filepaths
                                            tf.constant(y))) # labels
    data_batch = data.map(get_image_label).batch(BATCH_SIZE)

    return data_batch

  else:
    print("Creatin training data batches...")
    #Turn filpaths and labels into Tensors
    data=tf.data.Dataset.from_tensor_slices((tf.constant(X),
                                             tf.constant(y)))
    #Shuffling pathnames and labels before mapping image preocessor function is faster that than shuffling image
    data = data.shuffle(buffer_size=len(X))
  
    #Create (image, label) tuple (this also turn the image path into a preprocessing image )
    data_batch = data.map(get_image_label).batch(BATCH_SIZE)
    return data_batch

In [ ]:
#Create training and validation data batches

train_data = create_data_batches(X_train, y_train)
valid_data = create_data_batches(X_valid, y_valid ,valid_data=True)

In [ ]:
train_data.element_spec,valid_data.element_spec

## Visualizing Data Batches

Our data is now in batches, however, these can be a little hard to understand/comprehend, let's visualize them!


In [ ]:
import matplotlib.pyplot as plt

#Create a function for viewing images in a data batch
def show_25_images(images, labels):
  """
  Displays a plot of 25 images and their labels from data batch.
  """
  fig=plt.figure(figsize=(10,10))
  #loop through 25 (for display 25 images)
  for i in range(25):
    ax=plt.subplot(5,5,i+1)
    ax.imshow(images[i])
    #Add the image label as the title
    ax.set_title(Unique_breeds[labels[i].argmax()])

    #Turn the grid lines off
    plt.axis("off")

In [ ]:
train_images, train_labels = next(train_data.as_numpy_iterator())
train_images, train_labels

In [ ]:
len(train_images), len(train_labels)

In [ ]:
#Now let's visualize the data in a training batch
show_25_images(train_images, train_labels)

In [ ]:
#Now let's visualize our validation data
valid_images, valid_labels = next(train_data.as_numpy_iterator())
show_25_images(valid_images,valid_labels)

##Buliding a model

Before we building a model, there are a few things we nee to define:

* The input shape (our images shape, in the form of Tensors) to our model.
* The output shape (image labels, in the form of Tensors) of our model.
* The URL of the model we want to use.
  from Tensorflow Hub : https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/classification/4

In [ ]:
#Set input shape to the model
INPUT_SHAPE = [None, IMG_SIZE, IMG_SIZE, 3] # batch ,height, width, color channels

#Setup output shape of model
OUTPUT_SHAPE = len(Unique_breeds)

#Setup model URL from Tensorflow Hub
MODEL_URL = "https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/classification/4"


Now we've got our inputs, outputs, and model ready to go.
Let's put them togather into a keras deep learning model!

Knowing this, let's create a functio nwhich:
* Take the input shape, output shape and the model we've chosen as parameters.
* Define the layer in keras model in sequential fastion (do this first, then this, then that).
* Compiles the model (says it should be evaluated and improved).
* Building the model (tells the model the inputs shape it'll be getting).
* Return the model

In [ ]:
#Create a function which builds a Keras model
def create_model(input_shape=INPUT_SHAPE, output_shape=OUTPUT_SHAPE, model_url=MODEL_URL):
  print("Building model with : ",MODEL_URL)

  #Setup the model layers
  model =tf.keras.Sequential([
                              hub.KerasLayer(MODEL_URL), # Layer 1 (Input layer)
                              tf.keras.layers.Dense(units=OUTPUT_SHAPE, activation="softmax") # Layer 2 (output layer)
  ])

  # Compile the model
  model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                optimizer= tf.keras.optimizers.Adam(),
                metrics=["accuracy"]
                )
  
  #Build the model
  model.build(INPUT_SHAPE)

  return model

In [ ]:
model = create_model()
model.summary()

## Creating callbacks
 
 Callbacks are helper function a model can use during trainging to do such things as save it progress, check it progress or stop training if a model stops improving

 We'll create two callbacks, one for TensorBoard which helps track our model progress and another for early-stopping which prevents our model from training for too long.

 ### TensorBoard Callback
 To setup TensorBoard callback, we need to do 3 things:
 1. Load the TensorBoard norebook extension
 2. Create a TensorBoard callback which is able to save logs
 to directory and pass it to our model's `fit()` function
 3. Visualize our model training logs with the `%tensorboard` magic function (we'll do this after model training).

In [ ]:
#Load TensorBoard notebook extension
%load_ext tensorboard 

In [ ]:
import datetime

# Craete a function to build TensorBoard callback
def create_tensorboard_callback():
   # create a log directory for storing TensorBoard logs
   logdir= os.path.join("/content/drive/MyDrive/Dog-Vision/logs",
                        #Make it so the  log get tracked whenever we run an experiment
                        datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
                        )
   return tf.keras.callbacks.TensorBoard(logdir)

###Early stopping callback

Early stopping helps stop our model from overfitting by stopping  training if a certain evalution metric stops improving

In [ ]:
#create early stoping call back
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy",
                                                  patience=3)

## Training a model (on subset of data)

Our first model is only going to train on 1000 images, to make sure everything is working.

In [ ]:
NUM_EPOCHS = 100 #@param {type:"slider", min:10 , max:100 , step:10}

In [ ]:
#Cheak to make sur we are running on GPU
print("GPU" , "available" if tf.config.list_physical_devices("GPU") else " not availble")

Let's create a function which train a model

* Create a model using `create_model()`
* Setup a TensorBoard callbacks using 'create_tensorboard_callback()'
* Call the `fit()` function on our model passing it the training data, validation data, number of epochs to train for (`NUM_EPOCHS`) and callbacks  we'd like to use
* Return the model

In [ ]:
#Build a function to train and return a trained model

def train_model():
  """
  Train a given model returns the trained version.
  """

  #Create a model
  model = create_model()

  #Create new TensorBoard seesion everytime we train a model
  tensorboard = create_tensorboard_callback()

  #Fit the model to the data passing it the callbacks we created
  model.fit(x=train_data,
            epochs=NUM_EPOCHS,
            validation_data=valid_data,
            validation_freq=1,
            callbacks=[tensorboard, early_stopping])
  
  #Return the fitted model
  return model

In [ ]:
#Fit the moodel to the data
model=train_model()

###Checkin the TensorBoard logs

The TensorBoard fuction (%tensorboard) will access the logs directory we create earlier and visualize its content.

In [ ]:
%tensorboard --logdir /content/drive/MyDrive/Dog-Vision/logs

##Makeing and evaluting prediction using a train model

In [ ]:
#Make prediction on validation data (not used to train on)
predictions= model.predict(valid_data, verbose=1)
predictions

In [ ]:
predictions.shape

In [ ]:
#First prediction
index= 42
print(predictions[index])
print(f"Max value (probability of prediction) : {np.max(predictions[index])}")
print(f"Sum : {np.sum(predictions[index])}")
print(f"Index number : {predictions[index].argmax()}")
print(f"Predicted label : {Unique_breeds[predictions[index].argmax()]}")

Having the above functionality is great but we want to be able to do it at scale.
And it would be even better if we could see the image the prediction is being made on!

**Note :** Prediction probabilities are also known as confidence levels.

In [ ]:
#Turn prediction probabiltites into their respective label (easier to understand)

def get_pred_label(prediction_probabilities):
  """
  Turns an array of predictio nprobabilities into a label.
  """
  return Unique_breeds[np.argmax(prediction_probabilities)]

# Get a prediction label based on an array of prediction probabilities
pred_label = get_pred_label(predictions[25])
pred_label

Now since our validation data is still in batch dataset,
we'll have to unbatchify it to make prediction on the validation images and then compare those prediction to the validation laebls (truth labels).

In [ ]:
valid_data

In [ ]:
#Create a function to unbatch a batch dataset
def unbatchify(data):
  """
  Takes a batched dataset of (image, label) Tensors and returns separate arrays
  of the images and labels.
  """
  images = []
  labels =[]

  #Loop throug unbatch data
  for image,label in valid_data.unbatch().as_numpy_iterator():
    images.append(image)
    labels.append(Unique_breeds[np.argmax(label)])
  return images, labels

#Unbatchify validation data

val_images, val_labels = unbatchify(valid_data)

In [ ]:
val_images[0] , val_labels[0]

In [ ]:
get_pred_label(val_labels[34])

Now we've got ways to get:
* Predictino labels
* Validation labels(truth labels)
* Validation images

let's make some function to make these all a bit more
visaulize.

we'll create a function which:
* Take an array of prediction probabilities, an array of truth labels and an array of images and integers.
* Convert the prediction probabilities to a predicted label.
* Plot the predicted label, its predicted probability, the truth label and the target image on a single plot.

In [ ]:
def plot_pred(prediction_probabilities, labels, images, n=1):
  """
  View the prediction, ground truth and image for sample n
  """

  pred_prob, true_label, image =prediction_probabilities[n], labels[n], images[n]

  #Get the pred label 
  pred_label = get_pred_label(pred_prob)

  #Plot images & remove ticks
  plt.imshow(image)
  plt.xticks([])
  plt.yticks([])
  
  #Change the color of the title depending on if the prediction is right or wrong

  if pred_label == true_label:
    color="green"
  else:
    color="red"

  #Change plot title to be predicted, probability of prediction and truth label
  plt.title("prediction : {} \n pred_probability : {:.2f}% \n True_label : {}".format(pred_label, np.max(pred_prob)*100, true_label), color=color)
  

In [ ]:
plot_pred(prediction_probabilities=predictions,
          labels=val_labels,
          images=val_images,
          n=74)

Now've got one function to visualize our model top prediction, let's make another to view our model top 10 predictions

This function will:
* Take an input of prediction probabilities array and a ground thruth array and an integer
* Find the top 10:
  * Prediction probabilities indexes
  * Prediction probabilities values
  * Prediction labels
* Plot top 10 prediction probability values and labels, coloring the true label green

In [ ]:
def plot_pred_conf(prediction_probabilities, labels, n=1):
  """
  Plus the top 10 highest prediction confidence along with the truth label for sample n.
  """

  pred_prob, true_label =prediction_probabilities[n], labels[n]

  #Get the predicted label
  pred_label  = get_pred_label(pred_prob)

  #Find the top 10 prediction confidence indexes
  top_10_pred_indexes = pred_prob.argsort()[-10:][::-1]

  #Find the top 10 prediction confidence values
  top_10_pred_values = pred_prob[top_10_pred_indexes]

  #Find top_10_pred_labels
  top_10_pred_labels = Unique_breeds[top_10_pred_indexes]

  #Setup plot

  top_plot = plt.bar(np.arange(len(top_10_pred_labels)),
                     top_10_pred_values,
                     color="grey")
  
  plt.xticks(np.arange(len(top_10_pred_labels)),
             labels= top_10_pred_labels,
             rotation="vertical")
 

  

  #Change color of true label
  if np.isin(true_label, top_10_pred_labels):
    top_plot[np.argmax(top_10_pred_labels == true_label)].set_color("green")
  else:
    pass

In [ ]:
plot_pred_conf(prediction_probabilities=predictions,
               labels=val_labels,
               n=9)

Now we've got some function to help  us viusalize our predictino and evalute our model, Let's check out a few.

In [ ]:
#Let's check out few predictions and their different values

i_multiplier = 10
num_rows = 3
num_cols= 2
num_images = num_rows*num_cols
plt.figure(figsize=(10*num_cols , 5*num_rows))

for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_pred(prediction_probabilities=predictions,
            labels=val_labels,
            images=val_images,
            n=i+i_multiplier)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_pred_conf(prediction_probabilities=predictions,
                labels=val_labels,
                n=i+i_multiplier)
plt.tight_layout()



##Saving and reloading a trained Model


In [ ]:
#Create a function to save model
def save_model(model, suffix=None):
  """
  save a given model in a models directory and appends a suffix (string).
  """
  #create a model directory pathname with current time
  modeldir=os.path.join("/content/drive/MyDrive/Dog-Vision/models",datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

  model_path =modeldir + "-" + suffix + ".h5" #save format of model

  print(f"Saving model to : {model_path}")
  model.save(model_path)
  return model_path
  

In [ ]:
#Create a function to a train model
def load_model(model_path):
  """
  Loads a saved model from specified path.
  """
  print(f"Loading saved moodel from : {model_path}")
  model=tf.keras.models.load_model(model_path, custom_objects={"KerasLayer":hub.KerasLayer})

  return model

Now we've got fuction to save and load a trained model, let's make sure thry work!

In [ ]:
#Save our model trained on 1000 images
save_model(model, suffix="1000-images-mobilenetv2-Adam")

In [ ]:
#load a trained  model 
loaded_1000_image_model=load_model("/content/drive/MyDrive/Dog-Vision/models/20210320-110517-1000-images-mobilenetv2-Adam.h5")


In [ ]:
#Evaluate the pre_saved model
model.evaluate(valid_data)


In [ ]:

#Evaluate the loaded model
loaded_1000_image_model.evaluate(valid_data)

##Traing a model on the full data

In [ ]:
len(X), len(y)

In [ ]:
#Create a data batch with full data set
full_data = create_data_batches(X,y)

In [ ]:
full_data

In [ ]:
#Create a model for full model
full_model = create_model()

In [ ]:
#Create full model callbacks
full_model_tensorboard = create_tensorboard_callback()

full_model_early_stopping = tf.keras.callbacks.EarlyStopping(monitor="accuracy",
                                                             patience=3)

**Note :** Running the cell below will take little while
(Maybe up to 30 minutes for the first epoch) because the GPU we're using the runtime has to load all the images into memory.

In [ ]:
# fit the full model to the full data
full_model.fit(x=full_data,
               epochs=NUM_EPOCHS,
               callbacks=[full_model_tensorboard, full_model_early_stopping])

In [ ]:
save_model(full_model,suffix="full_data_trained_model_mobilenetv2_Adam")

In [ ]:
# Load in the full model
loaded_full_model = load_model("/content/drive/MyDrive/Dog-Vision/models/20210320-141158-full_data_trained_model_mobilenetv2_Adam.h5")

## Making prediction on test data

Since our model has been trained on images in tghe form of Tensor batches, to make predictions on the test data, we'll have to get it into same format.

Luckily we created `create_data_batches()` earlier which can take a list of filenames as input and conver them into Tensor batches.

To make predictions on the test data, we'll:
* Get the test images filenames
* convert the filenames into test data batches using  `create_data_batches()` and setting the `test_data` parameter to `True` (since the test data doestn't have labels).
* Make prediction array by passing the test batches to the `predict()` method called on our model

In [ ]:
#Load test images filenames

test_path = "/content/drive/MyDrive/Dog-Vision/test/"
test_filenames = [test_path + images for images in os.listdir(test_path)]

In [ ]:
test_filenames[:10]

In [ ]:
len(test_filenames)

In [ ]:
#Create test data batch
test_data = create_data_batches(test_filenames, test_data=True)

In [ ]:
test_data

**Note :** calling `predict()` on ourfull model and passing it the test data batch will take a long time to run (about ~ 1 hour)

In [ ]:
#Make prediction on test data batch using loaded full model
test_predictions = loaded_full_model.predict(test_data,verbose=1)

In [ ]:
test_predictions

In [ ]:
#Save prediction (numpy array) to csv file (for acces later)
np.savetxt("/content/drive/MyDrive/Dog-Vision/preds_array.csv", test_predictions, delimiter=",")

In [ ]:
#Load prediction (NumPy array) from csv file
test_predictions = np.loadtxt("/content/drive/MyDrive/Dog-Vision/preds_array.csv",delimiter=",")

In [ ]:
test_predictions

In [ ]:
test_predictions.shape

## Preparing test dataset prediction for Kaggle

Looking at the Kaggle sample submission, we find that it wants our prediction probability outputs in a DataFrame with ID and a column for each diffrent dog breed.
https://www.kaggle.com/c/dog-breed-identification/overview/evaluation

In [ ]:
#Create a pandas DataFrame with empty columns
pred_df=pd.DataFrame(columns=["id"] + list(Unique_breeds))

os.path.splitext() : https://www.geeksforgeeks.org/python-os-path-splitext-method/

In [ ]:
#Append test image ID's to predictions DataFrame
test_ids = [os.path.splitext(fname)[0] for fname in os.listdir(test_path)]
test_ids

In [ ]:
pred_df["id"] = test_ids

In [ ]:
pred_df

In [ ]:
pred_df[list(Unique_breeds)] = test_predictions

In [ ]:
pred_df.head()

In [ ]:
#Save our prediction dataframe to CSV for submission to kaggle
pred_df.to_csv("/content/drive/MyDrive/Dog-Vision/full_model_prediction.csv",index=False)

In [ ]:
#Make predictions on custom images
my_dog_path = "/content/drive/MyDrive/Dog-Vision/My dog (saint_bernard)/"
my_dog_image = [my_dog_path + fname for fname in os.listdir(my_dog_path)]

In [ ]:
#Turn image into batch datasets
my_dog_data = create_data_batches(my_dog_image, test_data =True)

In [ ]:
#make prediction on my dog data
my_dog_pred  = loaded_full_model.predict(my_dog_data)

In [ ]:
#prediction labe
Unique_breeds[np.argmax(my_dog_pred)]

In [ ]:
for image in my_dog_data.unbatch().as_numpy_iterator():
  plt.imshow(image)